In [ ]:
import json
import re
import requests
import yaml
import time

# Load configuration (ensure config.yaml has your Perplexity key under "Perplexity")
with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)

PERPLEXITY_API_KEY = config["Perplexity"]
def robust_extract_json(response_text):
    code_block_match = re.search(r"```(?:json)?\s*([\s\S]+?)\s*```", response_text)
    if code_block_match:
        candidate = code_block_match.group(1)
        try:
            return json.loads(candidate)
        except json.JSONDecodeError:
            pass
    start = response_text.find("{")
    end = response_text.rfind("}")
    if start != -1 and end != -1 and end > start:
        candidate = response_text[start:end+1]
        try:
            return json.loads(candidate)
        except json.JSONDecodeError:
            return None

    return None

def update_row(row_dict):
    prompt = "how is the weather in solothurn today?"
    payload = {
        "model": "sonar-reasoning-pro",
        "messages": [
            {
                "role": "system",
                "content": "you are very ironic and sarcastic, but you are also very helpful."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "max_tokens": 1000
    }
    headers = { "Authorization": f"Bearer {PERPLEXITY_API_KEY}", "Content-Type": "application/json"}
    try:
        response = requests.post(
            "https://api.perplexity.ai/chat/completions",
            json=payload,
            headers=headers
        )
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Request error: {e}")
        return row_dict

    try:
        response_data = response.json()
    except json.JSONDecodeError as e:
        print(f"JSON decode error in response: {e}")
        return row_dict

    if "choices" not in response_data or not response_data["choices"]:
        print("No choices found in response.")
        return row_dict

    raw_reply = response_data["choices"][0]["message"]["content"]
    print("Raw reply:", raw_reply)  # For debugging

    updated_data = robust_extract_json(raw_reply)
    if updated_data is None:
        print("Could not extract valid JSON; returning original data.")
        return row_dict

    return updated_data

result = update_row({})
print("Result:", result)


KeyboardInterrupt: 

In [52]:
import json
import re
import requests
import pandas as pd
import time
import yaml
import os

with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)

PERPLEXITY_API_KEY = config["Perplexity"]
def robust_extract_json(response_text):
    code_block_match = re.search(r"```(?:json)?\s*([\s\S]+?)\s*```", response_text)
    if code_block_match:
        candidate = code_block_match.group(1)
        try:
            return json.loads(candidate)
        except json.JSONDecodeError:
            pass

    start = response_text.find("{")
    end = response_text.rfind("}")
    if start != -1 and end != -1 and end > start:
        candidate = response_text[start:end+1]
        try:
            return json.loads(candidate)
        except json.JSONDecodeError:
            return None

    return None

def update_row(row_dict):
    prompt = ( "You are an expert in verifying pharmaceutical data, especially for the German market. "
        "Your task is to verify and update each field of the given row using the most current online information from reliable sources. "
        "Please check every column (e.g., Therapiegebiet, Indikation, Verabreichung, Firma, Wirkstoff, Markennamen, "
        "EMEA / FDA Registered, Registrierung CH erwartet im, etc.) using authoritative websites such as compendium.ch, Swissmedic, "
        "and other trusted sources. \n\n"
        "Hinweis: Die Excel-Daten sind in deutscher Sprache. Bitte nutzen Sie auch deutschsprachige Quellen, wo angebracht. \n\n"
        "IMPORTANT: If you use any online source to verify or update a field, include the URL (source link) in the 'Website' column and make sure these URLs actually exist and not outdated. "
        "If multiple sources are used, separate them with a semicolon and two blank spaces for its readablility. If no source was used or no update was made, leave the field unchanged. \n\n"
        "Only update a field if you have confirmed new, reliable information. Otherwise, return the original value. "
        "Return the updated row as a JSON object with the same keys and no additional commentary or text. \n\n"
        "Row data:\n" + json.dumps(row_dict, indent=2, default=str))
    payload = {
        "model": "sonar-pro", # cost for 10 rows: 0.5 USD
        "web_search_options": {"search_context_size": "high"},
        "temperature": 0.0,
        
        "messages": [ { "role": "system","content": (
                    "You are a highly knowledgeable and diligent expert in pharmaceutical data verification. "
                    "Your role is to verify each field of the given German data using up-to-date online sources. "
                    "Double-check every column, and only update fields when you are certain about the new data via your extensive research. "
                    "If you cannot verify a field, return the original value. Do not include any commentary in your reply." )},
            {"role": "user", "content": prompt }],
        "max_tokens": 8000}

    headers = {"Authorization": f"Bearer {PERPLEXITY_API_KEY}","Content-Type": "application/json"}

    try:
        response = requests.post(
            "https://api.perplexity.ai/chat/completions",
            json=payload,
            headers=headers
        )
        response.raise_for_status()  # Raises an HTTPError if the status is 4xx, 5xx
    except requests.RequestException as e:
        print(f"Request error: {e}")
        return row_dict

    try:
        response_data = response.json()
    except json.JSONDecodeError as e:
        print(f"JSON decode error in response: {e}")
        return row_dict

    if "choices" not in response_data or not response_data["choices"]:
        print("No choices found in response.")
        return row_dict

    raw_reply = response_data["choices"][0]["message"]["content"]
    #print("Raw reply:", raw_reply)

    updated_data = robust_extract_json(raw_reply)
    if updated_data is None:
        print("Could not extract valid JSON; returning original row.")
        return row_dict

    return updated_data

input_filename = "Pipeline.xlsx"
output_filename = "updated_Pipeline_tabelle_LE2_sonar_pro_highsearch.xlsx"
sheet_name = "Tabelle für LE2 2022"

df = pd.read_excel(input_filename, sheet_name=sheet_name)
updated_rows = []

for index, row in df.head(10).iterrows():
    row_dict = row.to_dict()
    print(f"Processing row {index+1}/{len(df.head(1))}...")
    updated_data = update_row(row_dict)
    updated_rows.append(updated_data)
    time.sleep(1)

updated_df = pd.DataFrame(updated_rows)
updated_df.to_excel(output_filename, index=False)
print(f"Updated data saved to {output_filename}")


Processing row 1/1...
Processing row 2/1...
Processing row 3/1...
Processing row 4/1...
Processing row 5/1...
Processing row 6/1...
Processing row 7/1...
Processing row 8/1...
Processing row 9/1...
Processing row 10/1...
Updated data saved to updated_Pipeline_tabelle_LE2_sonar_pro_highsearch.xlsx


In [ ]:
import json
import re
import pandas as pd
import openai
import time
import os
from openai import OpenAI
import yaml

with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)

client = OpenAI(api_key=config["OPENAI_KEY_TEST"],)

def extract_json(reply):
    match = re.search(r"```(?:json)?\s*([\s\S]*?)\s*```", reply)
    if match:
        return match.group(1)
    return reply

def update_row(row_dict):
    prompt = (
        "You are an expert in verifying and analyzing pharmaceutical data, specializing in identifying active pharmaceutical ingredients (Wirkstoffe) in Phase 3 clinical trials and updating pipeline products. "
        "Your task is to perform a comprehensive market analysis by processing the provided Excel row and verifying its information using only trusted, reliable sources. "
        "These sources include, but are not limited to, ClinicalTrials.gov, EMA, Swissmedic, FirstWord Pharma, Evaluate Pharma, Fierce Pharma, DAZ, and official manufacturer websites. "
        "Focus on checking for any updates in clinical trial phases, approvals, or other key changes. "
        "Add additional information if available, but do not remove any existing data. "
        " Please ensure that the format of the data remains consistent, especially for dates (dd/mm/yyyy). "
        "Return ONLY the updated row as a JSON object with the same keys, with no additional text or explanation. "
        "For every online resource/link used, include the URL in the 'Website' column (if multiple sources are used, separate them with a '&') - moreover please in brackets give the date of the webpage like for example: wwww.pharmazie/news/acti.com (2025). "
        "If no update is needed for a field, retain the original value.\n\n"
        "Row data:\n" + json.dumps(row_dict, indent=2, default=str))
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini-search-preview",
            messages=[
                {"role": "system", "content": "You are an expert in pharmaceutical market analysis specializing in active pharmaceutical ingredients in Phase 3 clinical trials. You verify and update pipeline product data by cross-checking reliable and trusted online sources such as ClinicalTrials.gov, EMA, Swissmedic, FirstWord Pharma, Evaluate Pharma, Fierce Pharma, DAZ, and manufacturer websites. Your analysis should be thorough, focused on current approvals and phase updates, and must rely solely on verified sources."},
                {"role": "user", "content": prompt}
            ],max_tokens=5000)
        reply = response.choices[0].message.content.strip()
        json_content = extract_json(reply)
        try:
            updated_row = json.loads(json_content)
        except json.JSONDecodeError as e:
            print("JSON decode error:", e)
            return row_dict 
        return updated_row

    except Exception as e:
        print("Error updating row:", e)
        return row_dict  
input_filename = "Pipeline.xlsx"
output_filename = "updated_data.xlsx"
sheet_name = "Tabelle für LE2 2022"  
df = pd.read_excel(input_filename, sheet_name=sheet_name)
updated_rows = []
for index, row in df.head(3).iterrows():
    row_dict = row.to_dict()
    print(f"Processing row {index+1}/{len(df)}...")
    updated_data = update_row(row_dict)
    updated_rows.append(updated_data)
    time.sleep(1) 
updated_df = pd.DataFrame(updated_rows)
updated_df.to_excel(output_filename, index=False)
print(f"Updated data saved to {output_filename}")


Processing row 1/264...
Processing row 2/264...
Processing row 3/264...
Updated data saved to updated_data.xlsx


In [43]:
from openai import OpenAI
import pandas as pd
import json
import os
from utils import extract_table_from_text
import yaml

with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)
client = OpenAI(api_key=config["OPENAI_KEY_TEST"],)
response = client.responses.create(model="gpt-4o",
  input = [
    {
        "role": "system",
        "content":  [
        {"type": "input_text",
         "text":
            "Du bist ein Assistent, der aus einem gegebenen Prompt eine Excel-Tabelle generiert und bist bei jeder Spalte und jeder Reihe sehr präzise und überprüfst deine Antwort selbständig. "
            "Du kannst eine vollständige Liste aller Moleküle und deren Indikationen erstellen und repetierst dich nicht sondern überprüfst immer wieder auf die Korrektheit deiner Antwort. "
            "eine FDA-Entscheidung (inklusive FDA Priority Review) erwartet wird. Die Liste soll nach Indikation "
            "und erwarteten Zulassungsdatum sortiert sein und die folgenden Spalten enthalten: Brandname, Wirkstoff, "
            "Produkteigenschaften, Applikationsformen (inkl. Darreichungsformen wie Pen, Fertigspritze, Filmtablette, Kapsel usw.), "
            "Lagerungsbedingungen (insbesondere ob Kühlung erforderlich ist), spezielle Patientengruppen, Informationen für "
            "Ärzte/Apotheken/Patienten, Wirkmechanismus, Kontraindikationen, Nebenwirkungen, Interaktionen, Schulungshinweise, "
            "und zugelassene Konkurrenzprodukte und die wichtigsten Webseiten die du zur erstellung der Tabelle genommen hast (in Abständen falls mehrere)." 
            "Du lieferst die Antwort ausschliesslich einer geordneten Tabelle - es ist auch oke wenn du zu etwas keine Informationen findest - bitte erwähne dies aber."
            " Zudem schaust du immer wieder nach ob deine Tabelle Duplikationen enthält und entfernst diese - wir wollen keine Duplikationen in der Tabelle haben."
            "Die Tabelle soll mindestens 50 Moleküle und deren Indikationen umfassen, die im Jahr 2025 eine FDA-Entscheidung erwarten - nur die Interessanten."}]
    }, {
        "role": "user",
        "content":[
        {
          "type": "input_text",
          "text": 
            "Erstelle vollständige Liste aller Moleküle von allen Indikationen für die im Jahr 2025 eine Entscheidung der US-amerikanischen "
            "Food and Drug Administration (FDA) erwartet wird inklusive FDA Priority Review. Die Liste ist nach Indikation und erwartetem Zulassungsdatum sortiert. "
            "Die Liste umfasst im Weiteren: Brandname, Wirkstoff, Produkteigenschaften, Applikationsformen (einschließlich Darreichungsformen wie Pen, Fertigspritze, "
            "Filmtablette, Kapsel usw.), Lagerungsbedingungen (insbesondere ob eine Kühlung erforderlich ist), spezielle Patientengruppen, Informationen für Ärzte, "
            "Apotheken und Patienten, inklusive Wirkmechanismus, Kontraindikationen, Nebenwirkungen, Interaktionen und Schulungshinweisen, zugelassene Konkurrenzprodukte, Wichtigste Webseiten."
            "Gib mir mindestens 50 Moleküle und deren Indikationen, die im Jahr 2025 eine FDA-Entscheidung erwarten, wenn bereits (heute; 8.April.2025) zugelassen oder nicht- bitte erwähne dies in einer seperaten Spalte (Zulassungsstatus). "
            "Bitte liefere als Antwort ausschliesslich einer geordneten Tabelle - es ist auch oke wenn du zu etwas keine Informationen findest - bitte erwähne dies aber."
            "Bitte foukussiere dich nur auf HOCH-Interessante Moleküle und deren Indikationen - und überprüfe auch Online deren Zulassungsstatus"}]
    }
],
text={"format": {"type": "text"}},
reasoning={},
tools=[{"type": "web_search_preview",
        "search_context_size": "high"}],
temperature=0.1,
max_output_tokens=12000,
top_p=1,
store=False
)


In [70]:
from openai import OpenAI
import pandas as pd
import json
import os
from utils import extract_table_from_text
import yaml

with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)
client = OpenAI(api_key=config["OPENAI_KEY_TEST"],)
response = client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "system",
            "content": [
                {"type": "input_text",
                 "text": (
                     "Du bist ein Experte für pharmazeutische Wirkstoffe und deren regulatorische Prozesse bei der FDA - vorallem Zulassungsdaten oder noch nicht Zulassungen müssen online nachgeschaut werden. "
                     "Deine Aufgabe ist es, eine sorgfältig recherchierte, präzise und umfassende Excel-Tabelle zu erstellen. "
                     "Die Tabelle enthält mindestens 50 hochinteressante Moleküle und deren Indikationen, für welche im Jahr 2025 eine Entscheidung "
                     "der FDA erwartet wird (einschließlich FDA Priority Review). "
                     "Die Tabelle muss strikt frei von Duplikaten sein, wobei du jeden Eintrag eigenständig auf Genauigkeit und Vollständigkeit prüfst. "
                     "Sortiere die Tabelle primär nach Indikation und anschließend nach erwartetem Zulassungsdatum. Die Tabelle enthält die folgenden Spalten: "
                     "Brandname, Wirkstoff, Indikation, Produkteigenschaften, Applikationsformen (inkl. Pen, Fertigspritze, Filmtablette, Kapsel etc.), "
                     "Lagerungsbedingungen (insbesondere Kühlungsbedarf), spezielle Patientengruppen, Informationen für Ärzte/Apotheken/Patienten, "
                     "Wirkmechanismus, Kontraindikationen, Nebenwirkungen, Interaktionen, Schulungshinweise,, aktueller Zulassungsstatus,Wichtigste Webseiten (Quelle), zugelassene Konkurrenzprodukte, "
                     "Falls bestimmte Informationen nicht auffindbar sind, weise dies explizit aus."
                 )}
            ]
        },
        {
            "role": "user",
            "content": [
                {"type": "input_text",
                 "text": (
                     "Bitte erstelle eine sorgfältig recherchierte Excel-Tabelle zu mindestens 50 hochinteressanten Molekülen, deren Indikationen und zugehörigen FDA-Entscheidungen "
                     "(inklusive FDA Priority Review), die im Jahr 2025 erwartet werden. Die Tabelle soll folgende Details umfassen: Brandname, Wirkstoff, Indikation, Produkteigenschaften, "
                     "Applikationsformen (Pen, Fertigspritze, Filmtablette, Kapsel usw.), Lagerungsbedingungen (inkl. notwendiger Kühlung), spezielle Patientengruppen, "
                     "Informationen für Ärzte, Apotheken und Patienten, Wirkmechanismus, Kontraindikationen, Nebenwirkungen, Interaktionen, Schulungshinweise, zugelassene Konkurrenzprodukte, "
                     "Wichtigste Webseiten (Quellen) und den aktuellen Zulassungsstatus (Stand: 8. April 2025; zugelassen oder nicht). "
                     "Sortiere die Liste nach Indikation und erwartetem Zulassungsdatum und stelle sicher, dass keinerlei Duplikate vorhanden sind. "
                     "Überprüfe den Zulassungsstatus und verfügbare Informationen sorgfältig online und weise explizit darauf hin, falls bestimmte Daten nicht gefunden werden können."
                 )}
            ]
        }
    ],
    text={"format": {"type": "text"}},
    reasoning={},
    tools=[{"type": "web_search_preview", "search_context_size": "high"}],
    temperature=0,
    max_output_tokens=12000,
    top_p=1,
    store=False
)

In [71]:
from utils import extract_table_from_text
df = extract_table_from_text(response.output_text)
df

,Brandname,Wirkstoff,Indikation,Produkteigenschaften,Applikationsformen,Lagerungsbedingungen,Spezielle Patientengruppen,Informationen für Ärzte/Apotheken/Patienten,Wirkmechanismus,Kontraindikationen,Nebenwirkungen,Interaktionen,Schulungshinweise,Zugelassene Konkurrenzprodukte,Wichtigste Webseiten (Quelle),Aktueller Zulassungsstatus
0,**Reproxalap**,Reproxalap,Trockene Augen,Modulator reaktiver Aldehydspezies (RASP),Augentropfen,Raumtemperatur,Keine spezifischen Gruppen,Anwendungshinweise für Patienten,"Bindet an freie Aldehyde, reduziert RASP-Level",Überempfindlichkeit,Augenreizung,Keine signifikanten bekannt,Anwendungstechnik,Künstliche Tränen,[Clinical Advisor](https://www.clinicaladvisor...,Nicht zugelassen (PDUFA-Datum: 2. April 2025)
1,**Cabometyx**,Cabozantinib,Neuroendokrine Tumoren,Tyrosinkinase-Inhibitor,Tabletten,Raumtemperatur,Erwachsene und Kinder ab 12 Jahren,Dosierungsanpassungen für Kinder,"Hemmt Tyrosinkinasen, reduziert Tumorwachstum",Überempfindlichkeit,"Diarrhö, Müdigkeit",CYP3A4-Inhibitoren,Überwachung der Leberfunktion,Keine spezifischen,[Wikipedia](https://en.wikipedia.org/wiki/Cabo...,Zugelassen (März 2025)
2,**Dupixent**,Dupilumab,Chronische spontane Urtikaria,Monoklonaler Antikörper,Fertigspritze,Kühlschrank (2-8°C),Ab 12 Jahren,Schulungsmaterialien für Selbstinjektion,Hemmt IL-4 und IL-13 Signalwege,Überempfindlichkeit,Reaktionen an der Injektionsstelle,Lebendimpfstoffe vermeiden,Selbstinjektionstechnik,Omalizumab,[Clinical Advisor](https://www.clinicaladvisor...,Nicht zugelassen (PDUFA-Datum: 18. April 2025)
3,**Elamipretid**,Elamipretid,Barth-Syndrom,Mitochondriales Peptid,Injektion,Kühlschrank (2-8°C),Erwachsene,Anwendungshinweise für Ärzte,"Bindet an Cardiolipin, verbessert Mitochondrie...",Überempfindlichkeit,"Übelkeit, Kopfschmerzen",Keine signifikanten bekannt,Überwachung der Herzfunktion,Keine spezifischen,[Clinical Advisor](https://www.clinicaladvisor...,Nicht zugelassen (PDUFA-Datum: 29. April 2025)
4,**STS101**,Dihydroergotamin,Migräne,Nasales Pulver,Nasenspray,Raumtemperatur,Erwachsene,Anwendungshinweise für Patienten,Agonist an Serotoninrezeptoren,"Überempfindlichkeit, Herz-Kreislauf-Erkrankungen","Übelkeit, Schwindel",CYP3A4-Inhibitoren,Anwendungstechnik,Sumatriptan,[Clinical Advisor](https://www.clinicaladvisor...,Nicht zugelassen (PDUFA-Datum: 30. April 2025)
5,**Blujepa**,Gepotidacin,Unkomplizierte Harnwegsinfektionen,Triazaacenaphthylen-Antibiotikum,Tabletten,Raumtemperatur,Frauen ab 12 Jahren,Anwendungshinweise für Patienten,Hemmt bakterielle Topoisomerasen,Überempfindlichkeit,"Übelkeit, Durchfall",Keine signifikanten bekannt,Einnahmehinweise,Nitrofurantoin,[Wikipedia](https://en.wikipedia.org/wiki/Gepo...,Zugelassen (März 2025)
6,**Cardamyst**,Etripamil,Paroxysmale supraventrikuläre Tachykardie,Kalziumkanalblocker,Nasenspray,Raumtemperatur,Erwachsene,Anwendungshinweise für Patienten,"Blockiert Kalziumkanäle, verlangsamt Herzfrequenz","Überempfindlichkeit, Hypotonie","Nasale Beschwerden, Schwindel",Keine signifikanten bekannt,Anwendungstechnik,Verapamil,[GoodRx](https://www.goodrx.com/drugs/news/upc...,Nicht zugelassen (PDUFA-Datum: 27. März 2025)
7,**Novavax COVID-19 Vaccine**,NVX-CoV2373,COVID-19,Proteinbasierter Impfstoff,Injektion,Kühlschrank (2-8°C),Erwachsene,Impfaufklärungsmaterialien,Induziert Immunantwort gegen SARS-CoV-2,Überempfindlichkeit,"Schmerzen an der Injektionsstelle, Müdigkeit",Keine signifikanten bekannt,Impfstoffhandhabung,mRNA-Impfstoffe,[GoodRx](https://www.goodrx.com/drugs/news/upc...,Nicht zugelassen (PDUFA-Datum: April 2025)
8,**Gomekli**,Mirdametinib,Neurofibromatose Typ 1,MEK1/2-Inhibitor,"Kapseln, Tabletten zur Suspension",Raumtemperatur,Erwachsene und Kinder ab 2 Jahren,Dosierungsanpassungen für Kinder,"Hemmt MEK1/2, reduziert Tumorwachstum",Überempfindlichkeit,"Hautausschlag, Durchfall",CYP3A4-Inhibitoren,Überwachung der Leberfunktion,Selumetinib,[Wikipedia](https://en.wikipedia.org/wiki/Mird...,Zugelassen (Februar 2025)
9,**Encelto**,Revakinagene taro

In [ ]:
# drop duplicates
df = df.drop_duplicates(subset=["Brandname", "Wirkstoff"])

(45, 16)
(41, 16)


In [73]:
df
# save to excel
df.to_excel("output6.xlsx", index=False)

In [34]:
import requests
import yaml
from pydantic import BaseModel
from typing import List, Optional

from typing import List
from pydantic import BaseModel

class AnswerFormat(BaseModel):
    brand_name: str
    active_ingredient: str
    therapy_area: str
    indication: str
    product_properties: str
    application_forms: List[str]
    storage_conditions: str
    special_patient_groups: Optional[str]
    information_for_doctors: str
    information_for_pharmacies: str
    information_for_patients: str
    mechanism_of_action: str
    contraindications: str
    side_effects: str
    interactions: str
    training_notes: str
    approved_competitor_products: List[str]
    important_websites: List[str]
    current_approval_status: str
    expected_approval_date: Optional[str]

class MoleculeList(BaseModel):
    molecules: List[AnswerFormat]


    
with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)
token = config["Perplexity"]
url = "https://api.perplexity.ai/chat/completions"

payload = {
    "model": "sonar-deep-research",  # sonar #sonar-deep-research
    "messages": [
        {
            "role": "user",
            "content": (
                "Bitte erstelle eine sorgfältig recherchierte Tabelle mit mindestens 10 hochinteressanten Molekülen, deren Indikationen und zugehörigen FDA-Entscheidungen "
                "(inklusive FDA Priority Review), die im Jahr 2025 erwartet werden. Die Tabelle soll folgende Details umfassen:"
                "Brandname, Wirkstoff, Therapiegebiet, Indikation, Produkteigenschaften, Applikationsformen (Pen, Fertigspritze, Filmtablette, Kapsel usw.), Lagerungsbedingungen "
                "(inkl. notwendiger Kühlung), spezielle Patientengruppen, Informationen für Ärzte, Apotheken und Patienten, Wirkmechanismus, Kontraindikationen, "
                "Nebenwirkungen, Interaktionen, Schulungshinweise, zugelassene Konkurrenzprodukte, wichtigste Webseiten (Quellen) und den aktuellen Zulassungsstatus "
                "(Stand: 8. April 2025; zugelassen oder nicht). Sortiere die Liste nach Indikation und erwartetem Zulassungsdatum und stelle sicher, dass "
                "keinerlei Duplikate vorhanden sind. Überprüfe den Zulassungsstatus und verfügbare Informationen sorgfältig online und weise explizit darauf hin, wenn "
                "bestimmte Daten nicht gefunden werden können."
            )}],
    "max_tokens": 8000,
    "temperature": 0.2,
    "web_search_options": {"search_context_size": "medium"},
    "search_domain_filter": ["swissmedic.ch", "firstwordpharma.com", "evaluatepharma.com","fiercepharma.com", #"ema.europa.eu""compendium.ch"
                             "daz.online", "pharmazeutische-zeitung.de","-drugs.com",  "-fda.gov", "clinicaltrials.gov"],
    "response_format": {
        "type": "json_schema",
        "json_schema": {"schema": MoleculeList.schema()},
    },

}

headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)


In [35]:
import pandas as pd
import json
response_json = response.json()
raw_content = response_json["choices"][0]["message"]["content"]
cleaned_content = re.sub(r"<think>.*?</think>", "", raw_content, flags=re.DOTALL).strip()
data = json.loads(cleaned_content)
molecules = data["molecules"]
df = pd.json_normalize(molecules)
df

,brand_name,active_ingredient,therapy_area,indication,product_properties,application_forms,storage_conditions,special_patient_groups,information_for_doctors,information_for_pharmacies,information_for_patients,mechanism_of_action,contraindications,side_effects,interactions,training_notes,approved_competitor_products,important_websites,current_approval_status,expected_approval_date
0,Alyftrek,Vanzacaftor/Tezacaftor/Deutivacaftor,Cystische Fibrose,CFTR-mutationsbedingte zystische Fibrose (ab 6...,Erstes einmal täglich einzunehmendes CFTR-Modu...,[Filmtablette],"Raumtemperatur (15-30°C), vor Feuchtigkeit sch...","Kinder ≥6 Jahre, CF-Patienten mit 31 zusätzlic...",Switch-Option von Trikafta bei besserer Verträ...,Genetischer Test zur Mutationsbestimmung erfor...,Einnahme unabhängig von Mahlzeiten möglich,CFTR-Kanal-Modulation durch Triple-Kombination,Schwere Leberfunktionsstörungen,"Kopfschmerzen, nasopharyngeale Entzündung (≥10%)",CYP3A4-Induktoren (Dosisanpassung erforderlich),Schulung zur Atemwegsclearance begleiten,"[Trikafta, Symdeko]",[https://www.fiercepharma.com/marketing/top-10...,Zugelassen am 15.12.2024 (vorverlegt),2024-12-15
1,Datroway,Datopotamab Deruxtecan,Onkologie,NSCLC (nicht-kleinzelliges Lungenkarzinom),TROP2-gerichtetes ADC mit Topoisomerase-I-Inhi...,[Intravenöse Infusion],"2-8°C (Originalverpackung), lichtgeschützt","EGFR-mutierte Patienten, Zweitlinientherapie",OS-Vorteil von 15.6 vs. 11.8 Monaten vs. Docet...,Spezialisierte Zytostatika-Herstellung erforde...,Regelmäßige Lungenfunktionskontrollen empfohlen,Antikörper-Wirkstoff-Konjugat (ADC),Schwere interstitielle Lungenerkrankung,"Stomatitis (67%), Alopezie (45%)",Keine bekannten pharmakokinetischen Interaktionen,Schulung zur Erkennung von Pneumonitis-Symptomen,[Trodelvy],[https://www.fiercepharma.com/marketing/top-10...,Zugelassen für Brustkrebs am 17.01.2025; NSCLC...,2025-07-12
2,Journavx,Suzetrigin,Schmerztherapie,Mittelstarke bis starke akute Schmerzen,Erster selektiver NaV1.8-Inhibitor,[Filmtablette],"Raumtemperatur, Originalblister",Patienten mit Opioid-Kontraindikationen,"Keine Atemdepression, geringeres Abhängigkeits...",CYP3A4-Interaktionen beachten (Grapefruitsaft-...,"Startdosis 100mg, dann 50mg alle 12h",Peripherer Natriumkanal-Blocker (NaV1.8),Gleichzeitige Einnahme starker CYP3A4-Hemmer,"Juckreiz (15%), CK-Erhöhung (8%)",CYP3A4-Inhibitoren erhöhen Exposition um 300%,Schulung zur differenzierten Schmerzdokumentation,[Hydrocodon/Paracetamol],[https://www.pharmazeutische-zeitung.de/us-zul...,Zugelassen am 03.02.2025,2025-02-03
3,Tolebrutinib,Tolebrutinib,Neurologie,Nicht-relabierende sekundär progrediente MS,Erster BTK-Inhibitor für MS,[Filmtablette],"15-30°C, trocken lagern",SPMS-Patienten ohne Schübe,31% Reduktion der Behinderungsprogression,Therapiekontinuität sicherstellen (tägl. Einna...,Regelmäßige Leberwertkontrollen erforderlich,Bruton-Tyrosinkinase-Hemmung,Aktive Hepatitis,Leberenzymerhöhung (4.2%),CYP3A4-Substrate (Dosisanpassung),Patientenmonitoring auf hepatische Symptome,[Ocrevus],[https://www.fiercepharma.com/marketing/top-10...,Breakthrough-Designation 12/2024; Erwartete Zu...,2025-09-30
4,Mazdutide,Mazdutide,Endokrinologie,Adipositas/ Typ-2-Diabetes,Erster dualer GLP-1/Glucagon-Rezeptoragonist,[Fertigpen],"2-8°C (ungeöffnet), Raumtemp. bis 4 Wochen",BMI ≥30 oder ≥27 mit Komorbiditäten,Phase-3-Daten: 15.4% Gewichtsreduktion über 52...,Kühlkette beim Transport beachten,Selbstinjektionstechnik schulen,Duale Rezeptoragonismus,MEN-2-Syndrom,"Übelkeit (33%), Diarrhö (21%)",Verzögerte Magenentleerung (orale Medikation),Schulung zur Injektionstechnik und Nebenwirkun...,"[Semaglutid, Tirzepatid]",[https://www.fiercepharma.com/marketing/top-10...,Phase-3 abgeschlossen; Erwartete FDA-Zulassung...,2025-12-15
5,Nipocalimab,Nipocalimab,Neurologie,Generalisierte Myasthenia gravis (gMG),Neonatal-Fc-Rezeptor (FcRn)-Antikörper,[Intravenöse Infusion],"2-8°C, vor Licht schützen",Refraktäre gMG-Patienten,Reduktion MG-ADL-Score um 4.2 Punkte vs. Place

In [36]:
df.to_excel("output_perplexity_deep_research.xlsx", index=False)

In [12]:
resposne_json= response.json()
import re
import json
import pandas as pd

#response_dict = json.loads(response.text)
response_json['choices'][0]['message']['content']


def extract_table_from_response(response):
    response_json = response.json()
    assistant_text = response_json["choices"][0]["message"]["content"]
    cleaned_text = re.sub(r'<think>.*?</think>', '', assistant_text, flags=re.DOTALL)

    lines = cleaned_text.splitlines()
    header_pattern = re.compile(r"^\|\s*Brandname\s*\|", re.IGNORECASE)

    def parse_row(line):
        cells = [cell.strip() for cell in re.split(r"\s*\|\s*", line.strip())]
        if cells and cells[0] == '':
            cells = cells[1:]
        if cells and cells[-1] == '':
            cells = cells[:-1]
        return cells

    all_data = []  
    global_header = None
    i = 0
    while i < len(lines):
        line = lines[i]
        if header_pattern.match(line):
            current_header = parse_row(line)
            if global_header is None:
                global_header = current_header
            elif current_header != global_header:
                print("DEBUG: Unterschiedlicher Header gefunden: ", current_header)
            i += 1
            if i < len(lines) and re.match(r"^\|[-\s|]+$", lines[i]):
                i += 1  
            while i < len(lines) and lines[i].strip() != "" and not header_pattern.match(lines[i]):
                if "|" in lines[i]:
                    row = parse_row(lines[i])
                    if len(row) == len(global_header):
                        all_data.append(row)
                    else:
                        print(f"DEBUG: Überspringe Zeile (Spalten: {len(row)} vs. Header: {len(global_header)}): {row}")
                i += 1
        else:
            i += 1

    if global_header is None:
        print("DEBUG: Keine Headerzeile gefunden!")
        print("DEBUG: Bereinigter Text:")
        print(cleaned_text)
        return None

    if not all_data:
        print("DEBUG: Keine passende Datenzeilen gefunden.")
        return None
    df = pd.DataFrame(all_data, columns=global_header)
    citations_list = response_json.get("citations", [])
    df["All_Citations"] = ", ".join(citations_list)
    
    return df

df = extract_table_from_response(response)
df


DEBUG: Keine Headerzeile gefunden!
DEBUG: Bereinigter Text:

{ "molecules": [ { "brand_name": "Alyftrek", "active_ingredient": "Vanza Triple (elexacaftor/tezacaftor/deutivacaftor)", "therapy_area": "Cystic Fibrosis", "indication": "Treatment of cystic fibrosis in patients aged 6 years and older", "product_properties": "CFTR modulator with improved efficacy over previous generations", "application_forms": ["Film-coated tablet"], "storage_conditions": "Room temperature (15-30°C)", "special_patient_groups": "Pediatric patients ≥6 years", "information_for_doctors": "Monitor liver function tests regularly", "information_for_pharmacies": "Dispense with patient education materials", "information_for_patients": "Take with fatty foods for optimal absorption", "mechanism_of_action": "CFTR protein potentiator/corrector combination", "contraindications": "Hypersensitivity to components", "side_effects": "Headache, nasal congestion, elevated liver enzymes", "interactions": "Strong CYP3A inhibitors 

In [52]:
# save df to excel
df.to_excel("output_deepresearch_deepseekr1.xlsx", index=False)

In [ ]:

client = OpenAI(api_key=config["DEEPSEEK"],base_url="https://api.deepseek.com")

response = client.chat.completions.create(
  model="deepseek-reasoner",
  max_tokens=8000,
  messages=[{"role": "user","content":"Erstelle vollständige Tabelle aller Moleküle von allen Indikationen für die im Jahr 2025 eine Entscheidung der US-amerikanischen Food and Drug Administration (FDA) erwartet wird inklusive FDA Priority Review. Die Liste ist nach Indikation und erwartetem Zulassungsdatum sortiert. Die Liste umfasst im Weiteren: Brandname, Wirkstoff, Produkteigenschaften, Applikationsformen (einschließlich Darreichungsformen wie Pen, Fertigspritze, Filmtablette, Kapsel usw.), Lagerungsbedingungen (insbesondere ob eine Kühlung erforderlich ist), spezielle Patientengruppen, Informationen für Ärzte, Apotheken und Patienten, inklusive Wirkmechanismus, Kontraindikationen, Nebenwirkungen, Interaktionen und Schulungshinweisen, zugelassene Konkurrenzprodukte\nAlles klar. Ich stelle eine vollständige Liste aller Arzneimittel (Moleküle) zusammen, für die im Jahr 2025 eine FDA-Entscheidung erwartet wird – einschließlich aller Indikationen und sowohl regulärer als auch Priority Reviews.\nDie Übersicht wird nach Indikation und erwartetem Zulassungsdatum sortiert. Zu jedem Produkt werde ich die folgenden Informationen angeben:\n·         Brandname und Wirkstoff\n·         Produkteigenschaften\n·         Applikationsformen (inkl. Darreichungsformen wie Pen, Fertigspritze, Filmtablette usw.)\n·         Lagerungsbedingungen (v. a. ob Kühlung erforderlich ist)\n·         Relevante Patientengruppen\n·         Informationen für Ärzte, Apotheken und Patienten\n·         Wirkmechanismus, Kontraindikationen, Nebenwirkungen, Interaktionen und Schulungshinweise\n·         Zugelassene Konkurrenzprodukte\nIch melde mich, sobald ich die Ergebnisse für dich zusammengestellt habe."
        }
      ],
  stream =False) 
print(response.choices[0].message.content)

**Hinweis:** Die folgende Tabelle ist ein **Beispiel** basierend auf öffentlich zugänglichen Daten bis Oktober 2023. Da viele FDA-Entscheidungen für 2025 noch nicht finalisiert sind, werden **Platzhalter** für unbestätigte Informationen verwendet. Aktualisierungen erfordern eine Überprüfung durch aktuelle Quellen.

---

### **FDA-Entscheidungen 2025: Arzneimittelliste (nach Indikation und Datum sortiert)**  
| **Indikation**                     | **Erwartetes Zulassungsdatum** | **Brandname (Wirkstoff)**             | **Produkteigenschaften**                                                                 | **Applikationsformen**        | **Lagerungsbedingungen**           | **Relevante Patientengruppen**                | **Informationen für Ärzte/Apotheken/Patienten**                                                                 | **Wirkmechanismus, Kontraindikationen, Nebenwirkungen, Interaktionen, Schulung**                                                                          

In [ ]:
from openai import OpenAI
from openai import OpenAI
import pandas as pd
import json
from dotenv import load_dotenv
import os
import yaml

load_dotenv()

with open("config.yaml", "r") as file:
     config = yaml.safe_load(file)


client = OpenAI(api_key=config["OPENAI_KEY_TEST"],)




response = client.chat.completions.create(
  model="gpt-4o",
  max_tokens=16384,
  messages=[
    {
      "role": "system",
      "content": [
        {
          "text": "You are a pharmaceutical data expert and data extraction specialist. Your task is to compile a comprehensive list of all drug molecules for which an FDA decision is expected in 2025, including those with FDA Priority Review. This list must be detailed and organized by indication and expected approval date.\n\nGather the following details for each molecule:\n\n- **Indication**\n- **Expected Approval Date**\n- **Brand Name and Active Ingredient**\n- **Product Attributes**\n- **Application Forms** (e.g., pen, prefilled syringe, film-coated tablet, capsule, etc.)\n- **Storage Conditions** (especially whether cooling is required)\n- **Relevant Patient Groups**\n- **Information for Doctors, Pharmacies, and Patients**\n- **Mechanism of Action, Contraindications, Side Effects, Interactions, and Training Notes**\n- **Approved Competitive Products**\n- **Sources**\n\nEnsure accuracy by cross-checking data from reputable sources and note any uncertainties encountered during the information collection process.\n\n# Steps\n\n1. Extract a list of molecules with an FDA decision expected in 2025.\n2. Organize the data by indication and expected approval date.\n3. Collect and compile detailed information for each molecule as listed above.\n4. Use placeholders for data specific to individual drugs where necessary.\n5. Validate the collected data against credible sources to ensure accuracy.\n\n# Output Format\n\nPresent the results in a detailed table format, where each row represents a molecule, and the columns align with the aforementioned data fields.\n\n# Examples\n\n- **Indication:** [Example Indication]\n- **Expected Approval Date:** [Date]\n- **Brand Name and Active Ingredient:** [Brand Name] (Active Ingredient)\n- **Product Attributes:** [Attributes]\n- **Application Forms:** [Forms]\n- **Storage Conditions:** [Conditions]\n- **Relevant Patient Groups:** [Groups]\n- **Information for Doctors, Pharmacies, Patients:** [Information]\n- **Mechanism of Action, Contraindications, Side Effects, Interactions, and Training Notes:** [Details]\n- **Approved Competitive Products:** [Products]\n- **Sources:** [Source Information]\n\n(Note: These are placeholders. Real examples should be completed with accurate and specific data.)\n\n# Notes\n\n- Pay particular attention to any molecules marked for FDA Priority Review, as these often have implications for earlier approval.\n- If data discrepancies arise, ensure to document sources and note the details of any uncertainties.\n- Ensure that the information is up-to-date and reflects the most recent submissions and reviews from the FDA.",
          "type": "text"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "text": "Create a complete table of all molecules for all indications for which a decision by the US Food and Drug Administration (FDA) is expected in 2025, including FDA Priority Review. The list is sorted by indication and expected approval date. Furthermore, the list includes: brand name, active ingredient, product properties, forms of application (including dosage forms such as pen, prefilled syringe, film-coated tablet, capsule, etc.), storage conditions (especially whether cooling is required), special patient groups, and information for physicians, pharmacies, and patients, including mechanism of action, contraindications, side effects, interactions and training instructions, and approved competitor products.\nAll right. I will compile a complete list of all medications (molecules) for which an FDA decision is expected in 2025 – including all indications and both regular and Priority Reviews.\nThe overview will be sorted by indication and expected approval date. For each product, I will provide the following information: · Brand name and active ingredient · Product properties · Forms of application (including dosage forms such as pen, prefilled syringe, film-coated tablet, etc.) · Storage conditions (especially whether cooling is required) · Relevant patient groups · Information for physicians, pharmacies, and patients · Mechanism of action, contraindications, side effects, interactions, and training instructions · Approved competitor products\nI will get back to you as soon as I have compiled the results for you.",
          "type": "text"
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "For example :  As of March 31, 2025, the U.S. Food and Drug Administration (FDA) has approved several novel drugs and vaccines, with additional decisions anticipated throughout the year. Below is a comprehensive table of these medications, organized by indication and expected approval date. Each entry includes detailed information on the brand name, active ingredient, product properties, forms of application, storage conditions, special patient groups, and pertinent information for healthcare providers and patients.\n\n| Indication | Expected Approval Date | Brand Name | Active Ingredient | Product Properties | Forms of Application | Storage Conditions | Special Patient Groups | Mechanism of Action | Contraindications | Side Effects | Interactions | Training Instructions | Approved Competitor Products |\n|------------|------------------------|------------|-------------------|--------------------|----------------------|--------------------|------------------------|---------------------|-------------------|--------------|-------------|----------------------|------------------------------|\n| **Tenosynovial Giant Cell Tumor** | February 14, 2025 | Romvimza | Vimseltinib | Switch-control kinase inhibitor targeting CSF1R | Oral capsules | Store at room temperature | Adults with symptomatic tenosynovial giant cell tumor | Inhibits CSF1R to reduce tumor growth | Hypersensitivity to vimseltinib | Fatigue, nausea, liver enzyme elevation | Potential interactions with CYP3A4 inhibitors | Patients should be monitored for liver function | None approved |\n| **Neurofibromatosis Type 1 with Symptomatic Plexiform Neurofibromas** | February 11, 2025 | Gomekli | Mirdametinib | MEK inhibitor | Oral capsules | Store at room temperature | Pediatric and adult patients with NF1 and inoperable tumors | Inhibits MEK1/2 to reduce tumor size | Hypersensitivity to mirdametinib | Rash, diarrhea, elevated creatine phosphokinase | Avoid concomitant use with strong CYP3A4 inhibitors | Regular monitoring of cardiac function recommended | None approved |\n| **Moderate to Severe Acute Pain** | January 30, 2025 | Journavx | Suzetrigine | Non-opioid analgesic, sodium channel blocker | Oral tablets (50 mg) | Store at room temperature | Adults requiring short-term pain management | Selectively inhibits NaV1.8 sodium channels to block pain signals | Hypersensitivity to suzetrigine | Dizziness, headache, gastrointestinal discomfort | Caution with other CNS depressants | Patients should be advised on potential CNS effects | Opioid analgesics (e.g., hydrocodone) |\n| **Acute Myeloid Leukemia and Myelodysplastic Syndrome** | January 21, 2025 | Grafapex | Treosulfan | Alkylating agent used in conditioning regimens | Intravenous infusion | Refrigerate; protect from light | Patients undergoing allogeneic hematopoietic stem cell transplantation | Cross-links DNA to inhibit cell division | Hypersensitivity to treosulfan | Myelosuppression, mucositis, hepatotoxicity | Increased toxicity with other myelosuppressive agents | Healthcare providers should monitor blood counts and organ function | Busulfan |\n| **Unresectable or Metastatic HR-Positive, HER2-Negative Breast Cancer** | January 17, 2025 | Datroway | Datopotamab deruxtecan-dlnk | Antibody-drug conjugate targeting TROP2 | Intravenous infusion | Refrigerate; protect from light | Adults with prior endocrine-based therapy and chemotherapy | Delivers cytotoxic agent to TROP2-expressing cancer cells | Hypersensitivity to components | Nausea, neutropenia, interstitial lung disease | Caution with other immunosuppressants | Patients should be monitored for pulmonary symptoms | Sacituzumab govitecan |\n| **Meningococcal Disease Prophylaxis** | February 14, 2025 | Penmenvy | Meningococcal groups A, B, C, W, and Y vaccine | Lyophilized powder for injection | Intramuscular injection | Refrigerate; do not freeze | Individuals aged 10 through 25 years | Induces immunity against Neisseria meningitidis serogroups A, B, C, W, and Y | Severe allergic reaction to vaccine components | Injection site reactions, fever, headache | None significant | Healthcare providers should be prepared for anaphylaxis management | Menveo, Bexsero |\n| **Chikungunya Disease Prevention** | February 14, 2025 | Vimkunya | Chikungunya vaccine, recombinant | Injectable suspension | Intramuscular injection | Refrigerate; do not freeze | Individuals aged 12 years and older | Induces immunity against chikungunya virus | Severe allergic reaction to vaccine components | Injection site reactions, fever, myalgia | None significant | Patients should be monitored for allergic reactions post-vaccination | None approved |\n| **Diabetes Mellitus** | February 14, 2025 | Merilog | Insulin aspart-szjj | Rapid-acting human insulin analog biosimilar to NovoLog | Subcutaneous injection | Refrigerate; do not freeze | Adults and pediatric patients with diabetes mellitus | Facilitates glucose uptake by cells | Hypoglycemia, hypersensitivity to insulin aspart | Hypoglycemia, weight gain, injection site reactions | Caution with other glucose-lowering agents | Patients should be trained on injection technique and hypoglycemia management | NovoLog |\n| **Uncomplicated Urinary Tract Infections** | March 26, 2025 (expected) | Gepotidacin | Gepotidacin | First-in-class oral antibiotic | Oral tablets | Store at room temperature | Female adults and adolescents with uUTIs | Inhibits bacterial DNA replication | Hypersensitivity to gepotidacin | Nausea, diarrhea, headache | Potential interactions with other antibiotics | Patients should complete the full course as prescribed | Nitrofurantoin, trimethoprim-sulfamethoxazole |\n| **Hereditary Angioedema Attacks** | June 17, 2025 (expected) | Sebetralstat | Sebetralstat | Oral plasma kallikrein inhibitor | Oral disintegrating tablets | Store at room temperature | Patients aged 12 years and older with HAE | Inhibits plasma kallikrein to reduce bradykinin production | Hypersensitivity to sebetralstat | Nausea, headache, dizziness | Caution with other kallikrein inhibitors | Patients should be trained on recognizing and managing HAE attacks | Icatibant (injection) |\n| **Vasomotor Symptoms Associated with Menopause** | July 26, 2025 (expected) | Elinzanetant | Elinzanetant | Dual NK-1 and NK-3 receptor antagonist | Oral tablets | Store at room temperature | Women experiencing moderate to severe VMS | Modulates KNDy neurons to reduce hot flashes | Hypersensitivity to elinzanetant | Nausea, headache, fatigue | Potential interactions with CYP3A4 inhibitors | Patients should be informed about the non-hormonal nature of the treatment | Hormone replacement therapy |\n| **H3 K27M-Mutant Diffuse Glioma** | August 18, 2025 (expected "
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "This is too short... it need to be logner-"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "retry: Erstelle vollständige Tabelle aller Moleküle von allen Indikationen für die im Jahr 2025 eine Entscheidung der US-amerikanischen Food and Drug Administration (FDA) erwartet wird inklusive FDA Priority Review. Die Liste ist nach Indikation und erwartetem Zulassungsdatum sortiert. Die Liste umfasst im Weiteren: Brandname, Wirkstoff, Produkteigenschaften, Applikationsformen (einschließlich Darreichungsformen wie Pen, Fertigspritze, Filmtablette, Kapsel usw.), Lagerungsbedingungen (insbesondere ob eine Kühlung erforderlich ist), spezielle Patientengruppen, Informationen für Ärzte, Apotheken und Patienten, inklusive Wirkmechanismus, Kontraindikationen, Nebenwirkungen, Interaktionen und Schulungshinweisen, zugelassene Konkurrenzprodukte\\nAlles klar. Ich stelle eine vollständige Liste aller Arzneimittel (Moleküle) zusammen, für die im Jahr 2025 eine FDA-Entscheidung erwartet wird – einschließlich aller Indikationen und sowohl regulärer als auch Priority Reviews.\\nDie Übersicht wird nach Indikation und erwartetem Zulassungsdatum sortiert. Zu jedem Produkt werde ich die folgenden Informationen angeben:\\n·         Brandname und Wirkstoff\\n·         Produkteigenschaften\\n·         Applikationsformen (inkl. Darreichungsformen wie Pen, Fertigspritze, Filmtablette usw.)\\n·         Lagerungsbedingungen (v. a. ob Kühlung erforderlich ist)\\n·         Relevante Patientengruppen\\n·         Informationen für Ärzte, Apotheken und Patienten\\n·         Wirkmechanismus, Kontraindikationen, Nebenwirkungen, Interaktionen und Schulungshinweise\\n·         Zugelassene Konkurrenzprodukte\\nIch melde mich, sobald ich die Ergebnisse für dich zusammengestellt habe."
        }
      ]
    }
  ],
  response_format={
    "type": "json_schema",
    "json_schema": {
      "name": "fda_drug_approvals_2025",
      "schema": {
        "type": "object",
        "required": ["molecules"],
        "properties": {"molecules": {"type": "array","items": {"type": "object","required": [
                "indication",
                "expected_approval_date",
                "brand_name",
                "active_ingredient",
                "product_properties",
                "competitor_products",
                "sources"],
        "properties": {
                "brand_name": {
                  "type": "string",
                  "description": "The brand name of the medication."
                },
                "indication": {
                  "type": "string",
                  "description": "The medical indication for which the drug is intended."
                },
                "sources": {
                  "type": "string",
                  "description": "Source urls for the information provided."
                },
                "active_ingredient": {
                  "type": "string",
                  "description": "The active ingredient contained in the medication."
                },
                "product_properties": {
                  "type": "object",
                  "required": [
                    "formulations",
                    "storage_conditions",
                    "special_patient_groups",
                    "info_for_healthcare"
                  ],
                  "properties": {
                    "formulations": {
                      "type": "array",
                      "items": {
                        "type": "string"
                      },
                      "description": "Different formulations of the product."
                    },
                    "storage_conditions": {
                      "type": "string",
                      "description": "Conditions under which the medication should be stored, including cooling requirements."
                    },
                    "info_for_healthcare": {
                      "type": "object",
                      "required": [
                        "mechanism_of_action",
                        "contraindications",
                        "side_effects",
                        "interactions",
                        "training_instructions"
                      ],
                      "properties": {
                        "interactions": {
                          "type": "array",
                          "items": {
                            "type": "string"
                          },
                          "description": "Known interactions with other drugs or substances."
                        },
                        "side_effects": {
                          "type": "array",
                          "items": {
                            "type": "string"
                          },
                          "description": "Potential adverse effects of the drug."
                        },
                        "contraindications": {
                          "type": "array",
                          "items": {
                            "type": "string"
                          },
                          "description": "Conditions or factors that would make the drug unsafe or inappropriate to use."
                        },
                        "mechanism_of_action": {
                          "type": "string",
                          "description": "How the drug works in the body."
                        },
                        "training_instructions": {
                          "type": "string",
                          "description": "Instructions for training on how to use the medication."
                        }
                      },
                      "description": "Information for healthcare professionals and patients.",
                      "additionalProperties": False
                    },
                    "special_patient_groups": {
                      "type": "array",
                      "items": {
                        "type": "string"
                      },
                      "description": "Groups of patients for whom special considerations may apply."
                    }
                  },
                  "description": "Properties of the product that are relevant for understanding its characteristics.",
                  "additionalProperties": False
                },
                "competitor_products": {
                  "type": "array",
                  "items": {
                    "type": "string"
                  },
                  "description": "Other approved products that serve as competitors."
                },
                "expected_approval_date": {
                  "type": "string",
                  "description": "The anticipated date of FDA approval."
                }
              },
              "additionalProperties": False
            },
            "description": "A list of all molecules for which FDA decisions are expected in 2025, sorted by indication and expected approval date."
          }
        },
        "additionalProperties": False
      },
      "strict": True
    }
  },
  reasoning={},
  tools=[{"type": "web_search"}],
  store=True
)


BadRequestError: Error code: 400 - {'error': {'message': "Missing required parameter: 'tools[0].function'.", 'type': 'invalid_request_error', 'param': 'tools[0].function', 'code': 'missing_required_parameter'}}

In [ ]:
response = response.choices[0].message.content
response




'{"molecules": [{"brand_name": "Vykat XR", "indication": "Prader-Willi-Syndrom", "sources": "https://www.reuters.com/business/healthcare-pharmaceuticals/us-fda-greenlights-first-ever-treatment-rare-metabolic-disorder-2025-03-26/", "active_ingredient": "Diazoxid-Cholin", "product_properties": {"formulations": ["Retardtabletten"], "storage_conditions": "Bei Raumtemperatur lagern", "info_for_healthcare": {"interactions": ["Vorsicht bei gleichzeitiger Anwendung mit blutdrucksenkenden Medikamenten"], "side_effects": ["Übelkeit", "Schwindel", "Hypoglykämie"], "contraindications": ["Überempfindlichkeit gegen Diazoxid"], "mechanism_of_action": "Aktiviert ATP-sensitive Kaliumkanäle, reduziert die Sekretion appetitanregender Neuropeptide", "training_instructions": "Patienten sollten über die Erkennung und Behandlung von Hypoglykämie informiert werden"}, "special_patient_groups": ["Kinder ab 4 Jahren mit Prader-Willi-Syndrom"]}, "competitor_products": ["Keine zugelassenen Konkurrenzprodukte"], "e

In [127]:
import demjson3
raw_output = response.strip()
if raw_output.startswith("```") and raw_output.endswith("```"):
    raw_output = raw_output.strip("```")
    if raw_output.lstrip().startswith("json"):
        raw_output = raw_output.lstrip()[len("json"):].strip()
print("First 500 characters of cleaned output:")
print(raw_output[:500])

First 500 characters of cleaned output:
{"molecules": [{"brand_name": "Vykat XR", "indication": "Prader-Willi-Syndrom", "sources": "https://www.reuters.com/business/healthcare-pharmaceuticals/us-fda-greenlights-first-ever-treatment-rare-metabolic-disorder-2025-03-26/", "active_ingredient": "Diazoxid-Cholin", "product_properties": {"formulations": ["Retardtabletten"], "storage_conditions": "Bei Raumtemperatur lagern", "info_for_healthcare": {"interactions": ["Vorsicht bei gleichzeitiger Anwendung mit blutdrucksenkenden Medikamenten"], 


In [130]:
import demjson3
import json

# Save raw output for inspection
with open("raw_output.txt", "w", encoding="utf-8") as f:
    f.write(raw_output)
print("Raw output saved to raw_output.txt")

try:
    # Try to parse with standard json first
    data = json.loads(raw_output)
except json.JSONDecodeError:
    # If that fails, use demjson3 which is more tolerant
    try:
        data = demjson3.decode(raw_output)
        print("Successfully parsed with demjson3")
    except demjson3.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        # Attempt to repair the JSON by adding closing brackets
        repaired_json = raw_output
        if '"molecules":' in raw_output and not raw_output.strip().endswith('}'):
            # Check if we're missing closing brackets for array and/or object
            if '[' in raw_output and ']' not in raw_output[-5:]:
                repaired_json += "]}"
            elif not repaired_json.strip().endswith('}'):
                repaired_json += "}"
            
            try:
                data = demjson3.decode(repaired_json)
                print("Successfully parsed after repairing JSON")
            except Exception as e:
                print(f"Failed to repair JSON: {e}")
                # Create a minimal valid structure to avoid errors in subsequent cells
                data = {"molecules": []}



Raw output saved to raw_output.txt
Error parsing JSON: String literal is not terminated
Failed to repair JSON: String literal is not terminated


In [131]:
# dict to an excel file 
import pandas as pd
df = pd.DataFrame(data["molecules"])
df.to_excel("FDA_2025_molecules.xlsx", index=False)

In [162]:
from openai import OpenAI
from pydantic import BaseModel
from typing import List
import pandas as pd
import json
from dotenv import load_dotenv
import os
import yaml

# Define Pydantic models with strict validation
class ApprovedCompetingProduct(BaseModel):
    product_name: str
    source_url: str

    class Config:
        extra = "forbid"

class InformationFor(BaseModel):
    doctors: str
    pharmacies: str
    patients: str

    class Config:
        extra = "forbid"

class ProductProperties(BaseModel):
    description: str
    application_forms: List[str]
    storage_conditions: str
    relevant_patient_groups: List[str]
    information_for: InformationFor
    mechanism_of_action: str
    contraindications: str
    side_effects: str
    interactions: str
    training_instructions: str
    approved_competing_products: List[ApprovedCompetingProduct]

    class Config:
        extra = "forbid"

class ProductOverview(BaseModel):
    brand_name: str
    product_properties: ProductProperties

    class Config:
        extra = "forbid"

# Load configuration (API key etc.)
load_dotenv()

with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)

client = OpenAI(api_key=config["OPENAI_KEY_TEST"])

# Call the API with the desired input and output format
response = client.responses.create(
    model="gpt-4o-2024-11-20",
    input=[
        {
            "role": "system",
            "content": [
                {
                    "type": "input_text",
                    "text": (
                        "You are a pharmaceutical data expert and data extraction specialist. Your task is to compile a comprehensive list of all drug molecules for which an FDA decision is expected in 2025, including those with FDA Priority Review. This list must be detailed and organized by indication and expected approval date.\n\n"
                        "Gather the following details for each molecule:\n\n"
                        "- **Indication**\n"
                        "- **Expected Approval Date**\n"
                        "- **Brand Name and Active Ingredient**\n"
                        "- **Product Attributes**\n"
                        "- **Application Forms** (e.g., pen, prefilled syringe, film-coated tablet, capsule, etc.)\n"
                        "- **Storage Conditions** (especially whether cooling is required)\n"
                        "- **Relevant Patient Groups**\n"
                        "- **Information for Doctors, Pharmacies, and Patients**\n"
                        "- **Mechanism of Action, Contraindications, Side Effects, Interactions, and Training Notes**\n"
                        "- **Approved Competitive Products**\n"
                        "- **Sources**\n\n"
                        "Ensure accuracy by cross-checking data from reputable sources and note any uncertainties encountered during the information collection process."
                    )
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "input_text",
                    "text": (
                        "Now give me a complete table of all molecules for all indications for which a decision by the US Food and Drug Administration (FDA) is expected in 2025, including FDA Priority Review. The list is sorted by indication and expected approval date. Furthermore, the list includes: brand name, active ingredient, product properties, forms of application (including dosage forms such as pen, prefilled syringe, film-coated tablet, capsule, etc.), storage conditions (especially whether cooling is required), special patient groups, and information for physicians, pharmacies, and patients, including mechanism of action, contraindications, side effects, interactions and training instructions, and approved competitor products.\n"
                    )
                }
            ]
        }
    ],
    text={
        "format": {
            "type": "json_schema",
            "name": "product_overview",
            "strict": True,
            "schema": ProductOverview.schema()
        }
    },
    reasoning={},
    tools=[{"type": "web_search"}],
    temperature=0.97,
    max_output_tokens=9030,
    top_p=1,
    store=True
)

if response.status == "incomplete" and response.incomplete_details.reason == "max_output_tokens":
    print("Ran out of tokens")
    if response.output_text:
        print("Partial output:", response.output_text)
    else:
        print("Ran out of tokens during reasoning")


C:\Users\FahRe\AppData\Local\Temp\ipykernel_12116\1510796883.py:102: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  "schema": ProductOverview.schema()


In [163]:
print(response.output_text)

{"brand_name":"Suzetrigine","product_properties":{"description":"Suzetrigine is a non-opioid pain medication developed for the treatment of moderate-to-severe acute pain. It functions as an oral, selective inhibitor of NaV1.8, a voltage-gated sodium channel that plays a critical role in pain signaling in the peripheral nervous system.","application_forms":["Oral tablet"],"storage_conditions":"Store at room temperature; no special cooling required.","relevant_patient_groups":["Adults with moderate-to-severe acute pain"],"information_for":{"doctors":"Suzetrigine offers a non-opioid alternative for managing acute pain, potentially reducing the risk of addiction associated with opioid use.","pharmacies":"Ensure proper dispensing and patient education regarding the non-opioid nature and administration of suzetrigine.","patients":"Suzetrigine is a non-opioid medication designed to manage acute pain without the risks associated with opioids."},"mechanism_of_action":"Suzetrigine selectively in

In [ ]:
import json
print(response.output_text)
print(json.dumps(response.output, default=lambda o: o.__dict__, indent=2))

[
  {
    "id": "ws_67ea9eb223248191bc5bb250f6bfcd320348dd1be5657452",
    "status": "completed",
    "type": "web_search_call"
  },
  {
    "id": "msg_67ea9eb50e248191955475841847cdc10348dd1be5657452",
    "content": [
      {
        "annotations": [],
        "text": "{\"brand_name\":\"Suzetrigine\",\"product_properties\":{\"description\":\"Suzetrigine is a non-opioid pain medication developed for the treatment of moderate-to-severe acute pain. It functions as an oral, selective inhibitor of NaV1.8, a voltage-gated sodium channel that plays a critical role in pain signaling in the peripheral nervous system.\",\"application_forms\":[\"Oral tablet\"],\"storage_conditions\":\"Store at room temperature; no special cooling required.\",\"relevant_patient_groups\":[\"Adults with moderate-to-severe acute pain\"],\"information_for\":{\"doctors\":\"Suzetrigine offers a non-opioid alternative for managing acute pain, potentially reducing the risk of addiction associated with opioid use.\",\"ph